In [1]:
import torch
import numpy as np
import pickle

from HMM_Likelyhood import HMM_log_likelihood,Optimizationloop, InitializeParameters,Accumulated_HHM_LL
with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = torch.from_numpy(X)
#X_tensor = torch.reshape(X_tensor.T,(-1,90,330))
torch.set_default_dtype(torch.float64)

In [25]:
# Initialize parameters to tune:

K=5
p=90
Subjects = 1

pi,kappa,mu,Tk = InitializeParameters(Subjects,p,K)

learning_rate = 2
n_iters = 100

torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':pi},
    {'params':kappa},
    {'params':mu},
    {'params':Tk},
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)




In [26]:
X_test = X_tensor[:,0:330*(Subjects+1)]
Pi_fit,Kappa_fit,mu_fit,Tk_fit = Optimizationloop(X_tensor,[pi,kappa,mu,Tk],lose=Accumulated_HHM_LL,Optimizer=Adam,n=Subjects,n_iters=n_iters,K=K)

epoch 1; Log-Likelihood = -16.428778749275704
epoch 2; Log-Likelihood = -70.2108018876068
epoch 3; Log-Likelihood = -75.68650442233469
epoch 4; Log-Likelihood = -77.06252170293347
epoch 5; Log-Likelihood = -78.6523913045426
epoch 6; Log-Likelihood = -80.41446741038993
epoch 7; Log-Likelihood = -82.26553674572818
epoch 8; Log-Likelihood = -84.21351104636062
epoch 9; Log-Likelihood = -86.10650849233687
epoch 10; Log-Likelihood = -88.10523531846349
epoch 11; Log-Likelihood = -90.08251951088454
epoch 12; Log-Likelihood = -92.07987472815638
epoch 13; Log-Likelihood = -94.09490816953185
epoch 14; Log-Likelihood = -96.10008946282233
epoch 15; Log-Likelihood = -98.14125104525962
epoch 16; Log-Likelihood = -100.14595284654355
epoch 17; Log-Likelihood = -102.1776309800921
epoch 18; Log-Likelihood = -104.21130534739794
epoch 19; Log-Likelihood = -106.21704361113969
epoch 20; Log-Likelihood = -108.2452832946941
epoch 21; Log-Likelihood = -110.24987674248396
epoch 22; Log-Likelihood = -112.25502335

In [27]:
Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

pi_est = Softmax(Pi_fit)
kappa_est = Softplus(Kappa_fit)
#kappa_est = Kappa_fit
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [28]:
print(Tk_fit[0])

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000]], grad_fn=<SelectBackward>)


In [29]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}: Pi = {pi_est[model]} , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0: Pi = 3.089896469440188e-14 , kappa = 1.3132616875182228


Model 1: Pi = 3.089896469440188e-14 , kappa = 1.3132616875182228


Model 2: Pi = 3.089896469440188e-14 , kappa = 1.3132616875182228


Model 3: Pi = 3.089896469440188e-14 , kappa = 1.3132616875182228


Model 4: Pi = 0.9999999999998765 , kappa = 177.43572279425118
